In [1]:
!pip install pandas scikit-learn requests beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [7]:
# Load phishing email dataset
emails = pd.read_csv("phishing email.csv")  # substitute actual filename
print("Email Dataset shape:", emails.shape)
print(emails.head())

# Standardize column names → assume `EmailText` and `Label`
emails.rename(columns={"email_text": "text", "label": "label"}, inplace=True)

# Load phishing URL dataset
urls = pd.read_csv("phishing URLs.csv")  # substitute actual filename
print("URL Dataset shape:", urls.shape)
print(urls.head())

# Standardize column names → assume `url` and `Label`
urls.rename(columns={"url": "text", "label": "label"}, inplace=True)

Email Dataset shape: (82486, 2)
                                       text_combined  label
0  hpl nom may 25 2001 see attached file hplno 52...      0
1  nom actual vols 24 th forwarded sabrae zajac h...      0
2  enron actuals march 30 april 1 201 estimated a...      0
3  hpl nom may 30 2001 see attached file hplno 53...      0
4  hpl nom june 1 2001 see attached file hplno 60...      0
URL Dataset shape: (54807, 2)
                                                 url      Type
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing
2                        https://kq0hgp.webwave.dev/  Phishing
3  https://brittishtele1bt-69836.getresponsesite....  Phishing
4         https://bt-internet-105056.weeblysite.com/  Phishing


In [9]:
print("Emails dataset columns:", emails.columns.tolist())
print("URLs dataset columns:", urls.columns.tolist())

Emails dataset columns: ['text_combined', 'label', 'source']
URLs dataset columns: ['text', 'Type', 'source']


In [10]:
# For emails dataset
emails.rename(columns={"text_combined": "text"}, inplace=True)

# For URLs dataset
urls.rename(columns={"Type": "label"}, inplace=True)

# Now combine
df = pd.concat(
    [emails[["text","label","source"]],
     urls[["text","label","source"]]],
    ignore_index=True
)

print("Combined dataset shape:", df.shape)
print(df.head())
print(df['label'].value_counts())

Combined dataset shape: (137293, 3)
                                                text label source
0  hpl nom may 25 2001 see attached file hplno 52...     0  email
1  nom actual vols 24 th forwarded sabrae zajac h...     0  email
2  enron actuals march 30 april 1 201 estimated a...     0  email
3  hpl nom may 30 2001 see attached file hplno 53...     0  email
4  hpl nom june 1 2001 see attached file hplno 60...     0  email
label
Phishing    54807
1           42891
0           39595
Name: count, dtype: int64


In [11]:
# Clean label values into consistent numeric form
df['label'] = df['label'].replace({
    "Phishing": 1, 
    "phishing": 1, 
    "malicious": 1,
    "legitimate": 0,
    "benign": 0,
    "good": 0
})

# Make sure labels are integers
df['label'] = df['label'].astype(int)

print(df['label'].value_counts())

label
1    97698
0    39595
Name: count, dtype: int64


C:\Users\syeds\AppData\Local\Temp\ipykernel_21860\1360821657.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({


In [13]:
X = df["text"]    # all email/URL text
y = df["label"]   # phishing = 1, safe = 0

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

Training samples: 109834
Testing samples: 27459


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=10000,    # top 10k words/bigrams
    ngram_range=(1,2),     # use unigrams + bigrams
    stop_words="english"   # remove common filler words
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)

X_train_tfidf shape: (109834, 10000)
X_test_tfidf shape: (27459, 10000)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train Logistic Regression
model = LogisticRegression(max_iter=2000)
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluation
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

✅ Accuracy: 0.9903492479697003

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98      7919
           1       0.99      0.99      0.99     19540

    accuracy                           0.99     27459
   macro avg       0.99      0.99      0.99     27459
weighted avg       0.99      0.99      0.99     27459



In [17]:
import pickle

# Save trained model
with open("phishing_combined_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save trained TF-IDF vectorizer
with open("combined_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("🎉 Model and vectorizer saved successfully!")

🎉 Model and vectorizer saved successfully!


In [18]:
# Load back the model
model = pickle.load(open("phishing_combined_model.pkl", "rb"))
vectorizer = pickle.load(open("combined_vectorizer.pkl", "rb"))

# Sanity check
test_cases = [
    "Congratulations! You have won $1000. Click here to claim!",  # phishing email
    "http://secure-login.paypai.com",                             # phishing URL
    "Your Amazon order is shipped successfully",                  # safe email
    "https://www.microsoft.com"                                   # safe URL
]

for case in test_cases:
    features = vectorizer.transform([case])
    prediction = model.predict(features)[0]
    print(f"{case} → {'🚨 Phishing' if prediction==1 else '✅ Safe'}")

Congratulations! You have won $1000. Click here to claim! → 🚨 Phishing
http://secure-login.paypai.com → 🚨 Phishing
Your Amazon order is shipped successfully → 🚨 Phishing
https://www.microsoft.com → 🚨 Phishing


In [14]:
# Extra Step: Save Lightweight Vectorizer Vocabulary for Deployment
#
# NOTE:
# - The original 'combined_vectorizer.pkl' file was very large (>100MB),
#   which cannot be uploaded easily to GitHub / Streamlit Cloud.
# - To fix this, we export only the vocabulary of the TF-IDF vectorizer 
#   as a JSON file. This drastically reduces file size (<10MB) and 
#   keeps deployment fast.
# - In 'app.py', we rebuild the TfidfVectorizer using this vocabulary.
#
# Output of this cell:
# 1. phishing_combined_model.pkl -> Trained Logistic Regression Model
# 2. vectorizer_vocab.json       -> TF-IDF Vocabulary (lightweight) 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle, json

#  Load datasets
emails = pd.read_csv("phishing email.csv")   # <-- change filename
urls   = pd.read_csv("phishing URLs.csv")    # <-- change filename

#  Print original columns
print("Emails dataset columns:", emails.columns.tolist())
print("URLs dataset columns:", urls.columns.tolist())

#  Rename columns to standard names
# For EMAILS dataset (usually 'text_combined' for email text, 'label' for class)
if "text_combined" in emails.columns:
    emails.rename(columns={"text_combined": "text"}, inplace=True)
elif "email_text" in emails.columns:
    emails.rename(columns={"email_text": "text"}, inplace=True)

if "Label" in emails.columns:
    emails.rename(columns={"Label": "label"}, inplace=True)

emails["source"] = "email"

# For URLS dataset (usually 'url' for domain, 'Type' or 'label' for class)
if "url" in urls.columns:
    urls.rename(columns={"url": "text"}, inplace=True)
if "Type" in urls.columns:
    urls.rename(columns={"Type": "label"}, inplace=True)
if "Label" in urls.columns:
    urls.rename(columns={"Label": "label"}, inplace=True)

urls["source"] = "url"

# 4. Concatenate both datasets
df = pd.concat(
    [emails[["text","label","source"]],
     urls[["text","label","source"]]],
    ignore_index=True
)

#  Normalize labels (so we only have 0=safe, 1=phish)
df['label'] = df['label'].replace({
    "Phishing": 1, "phishing": 1, "malicious": 1,
    "legitimate": 0, "benign": 0, "good": 0
})
df['label'] = df['label'].astype(int)

print("✅ Combined dataset ready:", df.shape)
print(df['label'].value_counts())
print(df.head())

#  Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, stratify=df["label"], random_state=42
)

#  Vectorizer
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
    stop_words="english"
)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf  = vectorizer.transform(X_test)

#  Logistic Regression
model = LogisticRegression(max_iter=2000)
model.fit(X_train_tfidf, y_train)
#  Save Model
with open("phishing_combined_model.pkl", "wb") as f:
    pickle.dump(model, f)

#  Save Vocabulary with Python int (convert from numpy.int32)
with open("vectorizer_vocab.json", "w") as f:
    json.dump({k:int(v) for k,v in vectorizer.vocabulary_.items()}, f)

print("🎉 Model saved as phishing_combined_model.pkl")
print("🎉 Vocabulary saved as vectorizer_vocab.json (lightweight)")

Emails dataset columns: ['text_combined', 'label']
URLs dataset columns: ['url', 'Type']
✅ Combined dataset ready: (137293, 3)
label
1    97698
0    39595
Name: count, dtype: int64
                                                text  label source
0  hpl nom may 25 2001 see attached file hplno 52...      0  email
1  nom actual vols 24 th forwarded sabrae zajac h...      0  email
2  enron actuals march 30 april 1 201 estimated a...      0  email
3  hpl nom may 30 2001 see attached file hplno 53...      0  email
4  hpl nom june 1 2001 see attached file hplno 60...      0  email


C:\Users\syeds\AppData\Local\Temp\ipykernel_2012\792455714.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({


🎉 Model saved as phishing_combined_model.pkl
🎉 Vocabulary saved as vectorizer_vocab.json (lightweight)
